# 模型增量更新与每日预测引擎 (批量版)

---

### **目标**
本 Notebook 是项目的**每日自动化运行脚本**。它会遍历配置文件中定义的所有股票，为每一只成功训练过模型的股票执行在线学习（热更新），并生成新的交易信号。

### **工作流程**
1.  **环境设置**: 导入库，加载配置。
2.  **主循环 (遍历股票)**: 对股票池中的每一只股票，执行以下步骤：
    a. **加载模型**: 加载该股票的基础模型 (LGBM, LSTM) 和融合模型 (Fuser)。如果任何模型缺失，则跳过该股票。
    b. **获取增量数据**: 智能地获取从上次批量训练至今的所有新数据。
    c. **执行增量训练**: 批量生成“历史预测”与“真实标签”，对融合模型进行 `partial_train`。
    d. **生成今日新预测**: 使用更新后的融合模型，生成用于次日交易的决策建议。

## 1. 环境设置与导入

In [1]:
# 文件路径: Incremental_Update.ipynb

import sys
from pathlib import Path

# --- 1. 动态导入 main_train 核心引擎 ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from main_train import run_load_config_and_modules, run_periodic_retraining_workflow
    print("--- INFO: 自动化再训练调度器 (Incremental Update) 已初始化。 ---")
except ImportError as e:
    raise ImportError(f"FATAL: 无法从 main_train.py 导入核心函数: {e}")

# --- 2. 加载全局配置和模块 ---
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

# --- 3. 执行周期性再训练工作流 ---
# 这个工作流会在内部自动将 end_date 更新为当天，并强制重新训练所有模型
if config and modules:
    print("\n" + "="*80)
    print("### 准备启动：周期性自动化再训练 ###")
    print(f"### 将会为配置文件中的 {len(config.get('stocks_to_process', []))} 只股票重新生成所有模型 ###")
    print("="*80 + "\n")
    try:
        run_periodic_retraining_workflow(config, modules)
        
        print("\n" + "*"*80)
        print("### 自动化再训练工作流成功执行完毕！ ###")
        print("*"*80)
    except Exception as e:
        print(f"\n--- ERROR: 在执行周期性再训练时发生严重错误 ---")
        print(e)
        import traceback
        traceback.print_exc()

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


--- INFO: 自动化再训练调度器 (Incremental Update) 已初始化。 ---
--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。

### 准备启动：周期性自动化再训练 ###
### 将会为配置文件中的 8 只股票重新生成所有模型 ###


--- ERROR: 在执行周期性再训练时发生严重错误 ---
run_periodic_retraining_workflow() missing 1 required positional argument: 'full_retrain'


Traceback (most recent call last):
  File "C:\Users\Akari\AppData\Local\Temp\ipykernel_28148\1244307769.py", line 31, in <module>
    run_periodic_retraining_workflow(config, modules)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
TypeError: run_periodic_retraining_workflow() missing 1 required positional argument: 'full_retrain'
